In [ ]:
# Setup
import nltk 
nltk.download('framenet_v17')

In [9]:
from app.src.frames.frame_checker_constuctor import FrameCheckerConstructor
from app.classes.grammar.selected_nodes.all_nodes import *

frame_checker = FrameCheckerConstructor.construct()

node_list = [
            RootNode('', 0),
            BeforeNode('', 0),
            EventNode('', 0),
            DomainEventNode('',0),
            DomainEventNameNode('', 0, 'occupy')
        ]

res = frame_checker.check_all_frames(node_list)

print(res)

[]


In [ ]:
# Get the template
from typing import Dict
from app.src.helpers.template_getter import get_template
contract = get_template('rental_t')

parameter_dict: Dict[str,str] = {}

contract.print_all_strings()

In [ ]:
#print(contract.to_sym())

In [ ]:
# User chooses the operation
ops_dict = {
    1: 'Customize a Parameter',
    2: 'Add a termination condition',
    3: 'Add a new domain event'
}

for opi in ops_dict:
    print(f'{opi} - {ops_dict[opi]}')
    
selected_op = int(input('Select an operation: '))


print(f'\nSelected operation: {ops_dict[selected_op]}\n')

In [ ]:
import nltk
from nltk.corpus import framenet as fn

from app.src.nlp.domain_event_creator import DomainEventCreator
from app.src.operations.dm_object_adder import DomainObjectAdder
from app.src.helpers.declarer import Declarer
from app.src.operations.declaration_adder import DeclarationAdder

if selected_op == '3':
    print('Adding a new domain event')
    dmc = DomainEventCreator(fn, contract.domain_model)
    domain_event = dmc.create()

    dm_adder = DomainObjectAdder()
    contract = dm_adder.add('events', contract, domain_event)

    print('\nInstantiating the event')
    declaration = Declarer.declare_input(contract, domain_event, 'events')

    # Will need to add to the declaration - separate operation
    declaration_adder = DeclarationAdder()
    contract = declaration_adder.add(contract, declaration)




In [ ]:
from app.src.operations.configs import ParameterConfig, ParameterSpec, OpCode
from app.src.operations.norm_adder import NormAdder
from app.classes.spec.prop_maker import PropMaker
from app.classes.spec.contract_spec import Power
from app.classes.spec.p_atoms import PAtomPredicateTrueLiteral
from app.classes.spec.power_function import PowerFunction, PFContract, PFContractName

# Adding a new termination power
if selected_op == '2':
    print('Adding termination condition')

    # Collect relevant user input (debtor, id)
    user_id = input('Enter an identifier: ')
    print('Choose which role has the power to terminate the contract:')

    roles = list(contract.domain_model.roles.keys())
    print(roles)
    for i,x in enumerate(roles):
        print(f'{i+1}: {x}')
    debtor_key = int(input('-- Enter #: ')) - 1

    debtor = roles[debtor_key]
    creditor = roles[1 - debtor_key]
    norm_id = f'termination_{user_id}'
    
    # Add the triggerless power
    new_power = Power(norm_id, None, debtor, creditor, PAtomPredicateTrueLiteral(), PFContract(PFContractName.Terminated))
    norm_adder = NormAdder()
    contract = norm_adder.add(contract, new_power)

    # Create the new spec
    parm_spec = ParameterSpec(
            op_codes=[OpCode.ADD_TRIGGER],
            configs = [
                ParameterConfig('powers', norm_id, 'trigger')
            ]
        )
    

In [ ]:
# Select the parameter
## TODO: Need a func to get the right parms (maybe include it with the contract collection)
#from app.templates.sample.t.nl_template import sample_parameters
from app.templates.rental_agreement.t.nl_template import parameters

if selected_op == '1':
    pki_dict = {i+1: pk for i,pk in enumerate(parameters)}

    for pki in pki_dict:
        print(f'{pki} - {pki_dict[pki]}')

    selected_pki = input('Select a parameter: ')
    selected_pk = pki_dict[int(selected_pki)]
    parm_spec = parameters[selected_pk]

    print(f'\nParameter: {selected_pk}')

In [ ]:
from app.src.grammar.grammar_generator import GrammarGenerator
from app.src.grammar.grammar_config import GrammarGeneratorConfig
from app.src.grammar.grammar_selector import GrammarSelector
from app.src.grammar.manual_node_selector import ManualGrammarNodeSelector
from app.src.grammar.helpers.domain_timepoint_extractor import DomainTimepointExtractor

domain_timepoint_extractor = DomainTimepointExtractor()
gg = GrammarGenerator(domain_timepoint_extractor)
grammar_config = GrammarGeneratorConfig(parm_spec.op_codes)
grammar_root = gg.generate(contract, grammar_config)

inner_selector = ManualGrammarNodeSelector()
gs = GrammarSelector(inner_selector)
selection = gs.select(grammar_root)


## Use Runner

In [ ]:
from app.src.runner import Runner

# Run for all parameters
for parm_config in parm_spec.configs:
    runner = Runner(contract, parm_config)
    contract = runner.update_contract(selection)

print(contract.to_sym())


## Step through

In [ ]:
from app.src.operations.helpers.default_event_getter import DefaultEventGetter

# Extract the default event
deg = DefaultEventGetter()

# Handle the case where there is none...
default_event = deg.get(contract.contract_spec, parm_config)

if default_event:
    print(f'Default Event: {default_event.to_sym()}')
else:
    print('No default event')

In [ ]:
#default_event = None

In [ ]:
from app.src.frames.frame_checker_constuctor import FrameCheckerConstructor

frame_checker = FrameCheckerConstructor.construct()
frames = frame_checker.check_all_frames(selection.nodes)

for x in frames:
    print(type(x), x.to_text())

main_frame = frames[0]
op_code = main_frame.op_code

In [ ]:
# Create the basket
from app.classes.grammar.selected_node import Basket
basket = Basket()
basket.default_event = default_event
basket.initial_norm = contract.contract_spec.__dict__[parm_config.norm_type][parm_config.norm_id]

In [ ]:
# Generate the Symboleo
symboleo_obj = selection.to_obj(basket)
print(symboleo_obj.to_sym())

In [ ]:
# Updating
from app.src.operations.parameter_updater_builder import ParameterUpdaterBuilder

parm_updater = ParameterUpdaterBuilder.build()

updated_contract = parm_updater.update(op_code, contract, symboleo_obj, parm_config)


In [ ]:
print(updated_contract.to_sym())